In [8]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
import random
import pandas as pd

In [9]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/"
}

url = 'https://markets.businessinsider.com/news/nvda-stock?p=1'

In [10]:
# soup = BeautifulSoup(page.text, 'lxml')
# articles = soup.find_all('div', class_ ='latest-news__story')

In [11]:
data = []
page_number = 1

while True:
    try:
        url = f'https://markets.businessinsider.com/news/nvda-stock?p={page_number}'

        page = requests.get(url, headers = headers)  

        time.sleep(random.uniform(3, 7))  

        soup = BeautifulSoup(page.text, 'lxml')
        articles = soup.find_all('div', class_ ='latest-news__story')

        if not articles:
            # Break the loop if no articles are found on the current page
            break

        for row in articles:
            datetime = row.find('time', class_= 'latest-news__date').get('datetime')
            title = row.find('a', class_= 'news-link').text
            source = row.find('span', class_= 'latest-news__source').text
            link = row.find('a',class_= 'news-link').get('href')
            data.append([datetime,title,source,link])
        
        page_number +=1
    
    except Exception as e:
        print(e)
        break



df = pd.DataFrame(data, columns = [ 'DateTime', 'Title', 'Source', 'Link'])

# df.to_csv('source.csv', index=False)


In [12]:
#df= pd.read_csv("source.csv")

In [13]:
from transformers import pipeline

pipe = pipeline("text-classification", model="ProsusAI/finbert", framework="pt")

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


In [14]:
def sentiment_analysis(text):
    data = pipe(text)
    return data

In [15]:
text = df['Title']

In [16]:
sentiment = []

for i in text:
        sentiment.append(sentiment_analysis(str(i)))

In [17]:
labels = [res[0]['label'] for res in sentiment]
scores = [res[0]['score'] for res in sentiment]

In [18]:
df['Label'] = labels
df['Score'] = scores

In [19]:
def sentiment_score(label):
    if label == 'positive':
        return 1
    if label == 'neutral':
        return 0
    if label == 'negative':
        return -1


In [20]:
df['Label'] = df['Label'].apply(sentiment_score)

In [21]:
df.to_csv('sentiment_score.csv', index = False)

In [22]:
# import pandas as pd 

# df = pd.read_csv('sentiment_score.csv')

In [23]:
# df.drop('Unnamed: 0', axis= 1, inplace=True)

In [24]:
df['Date'] = pd.to_datetime(df['DateTime'])
df['Date'] = df['Date'].dt.date

In [25]:
df.drop('DateTime', axis = 1, inplace=True)

In [26]:
df = df.set_index('Date')

In [27]:
def count_positive(value):

    if value == 1:
        return 1
    else:
        return 0

def count_neg(value):

    if value == -1:
        return 1
    else:
        return 0

def count_neu(value):

    if value == 0:
        return 1
    else:
        return 0


In [28]:
df['Positive'] = df['Label'].apply(count_positive)
df['Negative'] = df['Label'].apply(count_neg)
df['Neutral'] = df['Label'].apply(count_neu)

In [29]:
df_score = df.groupby(df.index).agg({
    'Score': 'mean',
    'Positive': 'sum',
    'Negative': 'sum',
    'Neutral': 'sum'
})

In [30]:
df_score['Total_Sentiment'] = df_score.apply(lambda row: row['Positive'] - row['Negative']/(row['Positive'] + row['Negative'] + row['Neutral']), axis=1)

In [31]:
df_score.tail()

,Score,Positive,Negative,Neutral,Total_Sentiment
Date,,,,,
2025-02-27,0.792029,17,11,13,16.731707
2025-02-28,0.840198,7,8,7,6.636364
2025-03-01,0.599051,0,0,1,0.000000
2025-03-02,0.820318,1,0,1,1.000000
2025-03-03,0.804065,0,2,5,-0.285714


In [32]:
print(df_score.duplicated().sum())

6


In [33]:
df_score.drop_duplicates(inplace=True)

In [34]:
import mysql.connector as connector

In [35]:
try:
    conn = connector.connect(user='root', password='123456', host='localhost', port='3306', database='my_db')
except connector.Error as er:
    print('Error code:', er.errorno)
    print('Error msg:', er.msg)

mycursor = conn.cursor()

In [36]:

mycursor.execute('SELECT * FROM stock_price')
price =  mycursor.fetchall()

In [37]:
df_price = pd.DataFrame(price, columns=['Date', 'Open',	'High',	'Low',	'Close',	'Volume',	'Dividends',	'Stock Splits'])

In [38]:
df_price.set_index('Date', drop=True)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1999-01-22,0.040122,0.044779,0.035585,0.037615,2.714690e+09,0.0,0.0
1999-01-25,0.040600,0.042033,0.037615,0.041556,5.104800e+08,0.0,0.0
1999-01-26,0.042033,0.042869,0.037734,0.038331,3.432000e+08,0.0,0.0
1999-01-27,0.038451,0.039406,0.036301,0.038212,2.443680e+08,0.0,0.0
1999-01-28,0.038212,0.038451,0.037854,0.038092,2.275200e+08,0.0,0.0
...,...,...,...,...,...,...,...
2025-02-24,136.560000,138.590000,130.080000,130.280000,2.513810e+08,0.0,0.0
2025-02-25,129.980000,130.200000,124.440000,126.630000,2.714290e+08,0.0,0.0
2025-02-26,129.990000,133.730000,128.490000,131.280000,3.225540e+08,0.0,0.0


In [39]:
print(df_price.duplicated().sum())

0


In [40]:
df_price.drop_duplicates(inplace=True)

In [41]:
df_merge = df_price.merge(df_score, on = 'Date')

In [42]:
df_merge.set_index('Date', drop=True, inplace=True)

In [43]:
df_merge.to_csv('test.csv')

In [44]:
len(df_merge)

1602

In [45]:
print(df_merge.index.duplicated().sum())

0


In [46]:
df_merge = df_merge[~df_merge.index.duplicated()]

In [47]:
print(df_merge.index.duplicated().sum())

0


In [48]:
mycursor.execute("SHOW TABLES")

tables = [table[0] for table in mycursor.fetchall()]

if 'price_sentiment' not in tables:
    mycursor.execute("CREATE TABLE price_sentiment (Date DATE UNIQUE, Open FLOAT(24), High FLOAT(24), Low FLOAT(24), Close FLOAT(24), Volume FLOAT(24), Dividends FLOAT(24), Stock_Splits FLOAT(24), Score FLOAT(24), Positive	INT, Negative INT,	Neutral INT, 	Total_Sentiment FLOAT(24))")
    print("Table 'stock_price' created successfully.")
else:
    print("Table 'stock_price' already exists.")


Table 'stock_price' created successfully.


In [49]:
sql = "INSERT IGNORE INTO  price_sentiment (Date, Open, High, Low, Close, Volume, Dividends, Stock_Splits, Score,	Positive,	Negative,	Neutral,	Total_Sentiment) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

In [50]:
tuples_list = list(df_merge.itertuples(index=True, name=None))


In [51]:
mycursor.executemany(sql, tuples_list)

In [52]:
conn.commit()

mycursor.close()
conn.close()